# Clasificación con Árboles de Decisión

En este notebook, abordamos un problema de clasificación multietiqueta utilizando árboles de decisión. Nuestro objetivo es predecir la probabilidad de vacunación contra el H1N1 y la gripe estacional. A continuación, describimos el proceso en detalle.

El algoritmo que vamos a utilizar es el árbol de decisión CART (Classification and Regression Trees). Lo aplicaremos con DecisionTreeClassifier de Scikit-learn que hace referencia al árbol de clasificación. Este árbol divide el espacio de características en regiones disjuntas mediante divisiones binarias (cada nodo tiene a lo sumo dos hijos).

## 1. Importar los datos

Cargamos los datos preprocesados y las etiquetas. El dataset ya ha sido tratado para manejar valores faltantes y aplicar codificaciones necesarias.

**Archivos utilizados:**

- preprocessed_training_simple_dataset.csv: Dataset preprocesado.
- training_set_labels.csv: Etiquetas correspondientes al dataset de entrenamiento.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

# Cargar los datos preprocesados y las etiquetas
X = pd.read_csv("../../Data/preprocessed_training_simple_dataset.csv", index_col="respondent_id")
y = pd.read_csv("../../Data/training_set_labels.csv", index_col="respondent_id", usecols=["respondent_id", "h1n1_vaccine", "seasonal_vaccine"])

## 2. Dividir los datos en conjuntos de entrenamiento y prueba

Para evaluar el modelo, dividimos los datos en conjuntos de entrenamiento y prueba. Usamos un 80% de los datos para entrenar y el 20% restante para probar.

In [3]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 3. Entrenar un árbol de decisión

Utilizamos el modelo DecisionTreeClassifier de Scikit-Learn y lo envolvemos en un MultiOutputClassifier para manejar el problema multietiqueta. Realizamos una búsqueda de hiperparámetros utilizando GridSearchCV.

In [4]:
# Modelo base: DecisionTreeClassifier
cart_model = DecisionTreeClassifier(random_state=42)

# Envolver en MultiOutputClassifier para clasificación multietiqueta
multi_cart = MultiOutputClassifier(cart_model)

In [5]:
# Espacio de búsqueda
param_grid = {
    'estimator__criterion': ['gini', 'entropy'],
    'estimator__max_depth': [3, 5, 10, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 5]
}

# Búsqueda de hiperparámetros
opt = GridSearchCV(multi_cart, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Entrenar el modelo con optimización
opt.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", opt.best_params_)
print("Mejor puntaje (AUROC):", opt.best_score_)

Mejores hiperparámetros: {'estimator__criterion': 'entropy', 'estimator__max_depth': 5, 'estimator__min_samples_leaf': 5, 'estimator__min_samples_split': 2}
Mejor puntaje (AUROC): 0.8345002438536696


## 4. Evaluar el modelo

Calculamos métricas de rendimiento clave, como precisión, recall, F1-score y nuestra métrica principal AUROC para ambas etiquetas.

In [6]:
# Predicciones de probabilidad
y_proba = opt.best_estimator_.predict_proba(X_test)

# AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test['h1n1_vaccine'], y_proba[0][:, 1])
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], y_proba[1][:, 1])

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Reporte de clasificación
y_pred = opt.best_estimator_.predict(X_test)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

AUROC para H1N1: 0.8516725323537758
AUROC para vacuna estacional: 0.8273459627937998
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.45      0.55      1135
           1       0.80      0.62      0.70      2488

   micro avg       0.77      0.57      0.65      3623
   macro avg       0.75      0.54      0.62      3623
weighted avg       0.77      0.57      0.65      3623
 samples avg       0.30      0.28      0.29      3623



c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VENV\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\USERS\PABLO\ONEDRIVE\DOCUMENTOS\ESTUDIOS\DATCOM\1ER CUATRIMESTRE\MINERÍA DE DATOS. PREPROCESAMIENTO Y CLASIFICACIÓN\FLUSHOTLEARNING\VEN

# 5. Visualizar el árbol de decisión

Visualizar el árbol nos ayuda a comprender cómo se toman las decisiones.

## 6. Generar el archivo de envío a la competición

Para generar el archivo de envío, aplicamos el modelo al dataset de prueba y creamos un archivo .csv con las predicciones.

In [ ]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_simple_dataset.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Realizar predicciones de probabilidad
y_proba_test = opt.best_estimator_.predict_proba(X_test_final)

# Extraer las probabilidades para la clase positiva (1)
y_proba_h1n1_test = y_proba_test[0][:, 1] 
y_proba_seasonal_test = y_proba_test[1][:, 1]

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("Submissions/DecisionTree_submission.csv", index=False)
print("Archivo 'DecisionTree_submission.csv' generado correctamente.")

Archivo 'DecisionTree_submission.csv' generado correctamente.
